In [ ]:
# set up the notebook
%load_ext autoreload
%autoreload 2

import sys
import os
import logging
logging.basicConfig(level=logging.ERROR)

from functools import partial
import re

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [5]:
import openai
import json

with open("./credential.json", "r") as f:
    credential = json.load(f)["openai"]
openai.api_key = credential

In [93]:
save_dict = {}

paper_index = 0

title = "Hardness in Markov Decision Processes: Theory and Practice"

abstract = """
"""

introduction = """
"""

methods = """
"""

experiments = """
"""

conclusion = """
"""

references = """
"""

appendix = """
"""

save_dict["paper_index"] = paper_index
save_dict["title"] = title
save_dict["abstract"] = abstract
save_dict["introduction"] = introduction
save_dict["methods"] = methods
save_dict["experiments"] = experiments
save_dict["conclusion"] = conclusion
save_dict["appendix"] = appendix
save_dict["references"] = references

In [94]:
# pilot system prompt: 
system_prompt = f"You are a computer science researcher currently reviewing a paper titled \"{title}\" for the NeurIPS computer science conference.

def turn_into_input_format(role, prompt):
    # role: "system", "user", or "assistant"
    return {"role": role, \
            "content": prompt}

system_prompt = turn_into_input_format("system", system_prompt)

In [99]:
question_section_mapping = {
    "1b": "conclusion",
    "1c": "conclusion",
    "2a": "methods",
    "2b": "methods",
    "3a": "experiments",
    "3b": "experiments",
    "3c": "experiments",
    "3d": "experiments",
    "4a": "experiments",
    "4b": "experiments",
    "4c": "experiments",
    "4d": "experiments",
    "4e": "experiments",
    "5a": "experiments",
    "5b": "experiments",
    "5c": "experiments"
}

In [100]:
def generate_prompts(question_section_mapping):

    q1b_prompt_1 = f'The following is the {question_section_mapping["1b"]} section of the paper you are reviewing:\n' + save_dict[question_section_mapping["1b"]] + \
                "\nBased on this section, do the authors describe limitations of their work?"

    q1b_prompt_2 = f'Here is the {question_section_mapping["1b"]} for the paper you are reviewing. Based on the contents in the {question_section_mapping["1b"]}, please answer whether the authors describe limitations of their work.\n{question_section_mapping["1b"]} section: ' + \
                save_dict[question_section_mapping["1b"]] + \
                "\nAnswer: "

    q1b_prompt_3 = f'{question_section_mapping["1b"]} section of the paper:\n' + save_dict[question_section_mapping["1b"]] + \
                f'\nBased on the {question_section_mapping["1b"]} section of the paper, do the authors describe any limitations of their work? Please quote a short portion of the section directly as evidence before answering.\nQuote: '

    q1c_prompt_1 = f'The following is the {question_section_mapping["1c"]} section of the paper you are reviewing:\n' + save_dict[question_section_mapping["1c"]] + \
                "\nBased on this section, do the authors address potential negative societal impacts of their work?"

    q1c_prompt_2 = f'Here is the {question_section_mapping["1c"]} for the paper you are reviewing. Based on the contents in the {question_section_mapping["1c"]}, please answer whether the authors address potential negative societal impacts in their work.\n{question_section_mapping["1c"]} section: ' + \
                save_dict[question_section_mapping["1c"]] + \
                "\nAnswer: "

    q1c_prompt_3 = f'{question_section_mapping["1c"]} section of the paper:\n' + save_dict[question_section_mapping["1c"]] + \
                f'\nBased on the {question_section_mapping["1c"]} section of the paper, do the authors discuss any potential negative societal impacts of their work? Please quote a short portion of the section directly as evidence before answering.\nQuote: '

    q2a_prompt_1_1 = "The following is the abstract of the paper you are reviewing:\n" + abstract + \
                    "\nBased on this abstract, do the authors have theoretical results? Please start your answer with \"Yes\", \"No\", or \"Unsure\"."

    q2a_prompt_1_2 = f'The following is the {question_section_mapping["2a"]} section of the paper you are reviewing:\n' + save_dict[question_section_mapping["2a"]] + \
                    "\nBased on this section, do the authors state the assumptions for their theoretical results?"

    q2a_prompt_2_1 = "The following is the introduction of the paper you are reviewing:\n" + introduction + \
                    "\nBased on the contents in the introduction, do the authors present theoretical results in the paper? Please start your answer with \"Yes\", \"No\", or \"Unsure\"."
    # note: a majority of the answers were just yes, no, or unsure. 

    q2a_prompt_2_2 = f'The following is the {question_section_mapping["2a"]} section of the paper you are reviewing:\n' + save_dict[question_section_mapping["2a"]] + \
                    "\nBased on this section, do the authors state the assumptions for their theoretical results?"

    q2a_prompt_3 = f'The following is the {question_section_mapping["2a"]} section of the paper you are reviewing:\n' + save_dict[question_section_mapping["2a"]] + \
                "\nBased on this section, do the authors state the assumptions for their theoretical results? If there are no theoretical results, please say so instead. "

    q2b_prompt_1_1 = "The following is the abstract of the paper you are reviewing:\n" + abstract + \
                    "\nBased on this abstract, do the authors have theoretical results? Please start your answer with \"Yes\", \"No\", or \"Unsure\"."

    q2b_prompt_1_2 = f'The following is the {question_section_mapping["2b"]} section of the paper you are reviewing:\n' + save_dict[question_section_mapping["2b"]] + \
                    "\nBased on this section, do the authors include complete proofs for all their theoretical results?"

    q2b_prompt_2_1 = "The following is the introduction of the paper you are reviewing:\n" + introduction + \
                    "\nBased on the contents in the introduction, do the authors present theoretical results in the paper? Please start your answer with \"Yes\", \"No\", or \"Unsure\"."

    q2b_prompt_2_2 = f'The following is the {question_section_mapping["2b"]} section of the paper you are reviewing:\n' + save_dict[question_section_mapping["2b"]] + \
                    "\nDo the authors include complete proofs for all their theoretical results?"

    q2b_prompt_3 = f'The following is the {question_section_mapping["2b"]} section of the paper you are reviewing:\n' + save_dict[question_section_mapping["2b"]] + \
                    "\nDo the authors include complete proofs for all their theoretical results? If there are no theoretical results, please say so instead."

    q3a_prompt_1 = f'The following is the {question_section_mapping["3a"]} section of the paper you are reviewing:\n' + save_dict[question_section_mapping["3a"]] + \
                "\nBased on this section, do the authors include code, data, and instructions needed to reproduce the main experimental results, or provide them in a URL?"

    q3a_prompt_2 = f'Here is the {question_section_mapping["3a"]} section for the paper you are reviewing. Based on the contents in the section, please answer whether the authors include code, data, and instructions needed to reproduce the main experimental results (either in the supplemental material or as a URL).\n{question_section_mapping["3a"]} section: ' + \
                save_dict[question_section_mapping["3a"]] + \
                "\nAnswer: "

    q3a_prompt_3 = f'{question_section_mapping["3a"]} section of the paper:\n' + save_dict[question_section_mapping["3a"]] + \
                f'\nBased on the {question_section_mapping["3a"]} section of the paper, do the authors include code, data, and instructions needed to reproduce the main experimental results (either in the supplemental material or as a URL)? Please quote a short portion of the section directly as evidence before answering.\nQuote: '

    q3b_prompt_1 = f'The following is the {question_section_mapping["3b"]} section of the paper you are reviewing:\n' + save_dict[question_section_mapping["3b"]] + \
                "Based on this section, do the authors specify all training details (e.g., data splits, hyperparameters, how they were chosen) for their results?"

    q3b_prompt_2 = f'Here is the {question_section_mapping["3b"]} section for the paper you are reviewing. Based on the contents in the section, please answer whether the authors specify all the training details (e.g., data splits, hyperparameters, how they were chosen).\n{question_section_mapping["3b"]} section: ' + \
                save_dict[question_section_mapping["3b"]] + \
                "\nAnswer: "

    q3b_prompt_3 = f'{question_section_mapping["3b"]} section of the paper:\n' + save_dict[question_section_mapping["3b"]] + \
                f'Based on the {question_section_mapping["3b"]} section of the paper, do the authors specify all the training details (e.g., data splits, hyperparameters, how they were chosen)? Please quote a short portion of the section directly as evidence before answering.\nQuote: '

    q3c_prompt_1 = f'The following is the {question_section_mapping["3c"]} section of the paper you are reviewing:\n' + save_dict[question_section_mapping["3c"]] + \
                "Based on this section, do the authors report error bars (e.g., with respect to the random seed after running experiments multiple times)?"

    q3c_prompt_2 = f'Here is the {question_section_mapping["3c"]} section for the paper you are reviewing. Based on the contents in the section, please answer whether the authors report error bars (e.g., with respect to the random seed after running experiments multiple times).\n{question_section_mapping["3c"]} section: ' + \
                save_dict[question_section_mapping["3c"]] + \
                "\nAnswer: "

    q3c_prompt_3 = f'{question_section_mapping["3c"]} section of the paper:\n' + save_dict[question_section_mapping["3c"]] + \
                f'Based on the {question_section_mapping["3c"]} section of the paper, do the authors report error bars (e.g., with respect to the random seed after running experiments multiple times)? Please quote a short portion of the section directly as evidence before answering.\nQuote: '

    q3d_prompt_1 = f'The following is the {question_section_mapping["3d"]} section of the paper you are reviewing:\n' + save_dict[question_section_mapping["3d"]] + \
                "Based on this section, do the authors include the amount of compute and type of resources used (e.g., type of GPUs, internal cluster, or cloud provider)?"

    q3d_prompt_2 = f'Here is the {question_section_mapping["3d"]} section for the paper you are reviewing. Based on the contents in the section, please answer whether the authors include the amount of compute and the type of resources used (e.g., type of GPUs, internal cluster, or cloud provider).\n{question_section_mapping["3d"]} section: ' + \
                save_dict[question_section_mapping["3d"]] + \
                "\nAnswer: "

    q3d_prompt_3 = f'{question_section_mapping["3d"]} section of the paper:\n' + save_dict[question_section_mapping["3d"]] + \
                f'\nBased on the {question_section_mapping["3d"]} section of the paper, do the authors include the amount of compute and the type of resources used (e.g., type of GPUs, internal cluster, or cloud provider)? Please quote a short portion of the section directly as evidence before answering.\nQuote: '

    q4a_prompt_1_1 = "The following is the methods section of the paper you are reviewing:\n" + methods + \
                    "\nBased on this section, do the authors use existing assets (e.g., code, data, models) that are not new contributions in their paper? Please start your answer with \"Yes\", \"No\", or \"Unsure\"."

    q4a_prompt_1_2 = f'The following is the {question_section_mapping["4a"]} section of the paper you are reviewing:\n{question_section_mapping["4a"]} section: ' + save_dict[question_section_mapping["4a"]] + \
                    f'\nBased on the section, do the authors cite the creators of the existing assets that they use?'

    q4a_prompt_2_1 = "The following is the methods section of the paper you are reviewing:\n" + methods + \
                    "\nBased on this section, do the authors use existing assets (e.g., code, data, models) that are not new contributions in their paper? Please start your answer with \"Yes\", \"No\", or \"Unsure\"."

    q4a_prompt_2_2 = f'The following is the {question_section_mapping["4a"]} section of the paper you are reviewing:\n{question_section_mapping["4a"]} section: ' + save_dict[question_section_mapping["4a"]] + \
                    f'\nBased on the section, do the authors cite the creators of the existing assets that they use? Please quote a short portion of each section directly as evidence before answering.\nQuote: '

    q4a_prompt_3 = f'The following is the {question_section_mapping["4a"]} section of the paper you are reviewing:\n{question_section_mapping["4a"]} section: ' + save_dict[question_section_mapping["4a"]] + \
                "\nBased on this section, if the authors are using existing assets (e.g., code, data, models), do they cite the creators of these existing assets?"

    q4b_prompt_1_1 = f'The following is the {question_section_mapping["4b"]} section of the paper you are reviewing:\n' + save_dict[question_section_mapping["4b"]] + \
                    "\nBased on this section, do the authors use existing assets (e.g., code, data, models) that are not new contributions in their paper? Please start your answer with \"Yes\", \"No\", or \"Unsure\"."

    q4b_prompt_1_2 = "\nBased on the section, do the authors mention the licenses of the existing assets that they use?"

    q4b_prompt_2_1 = f'The following is the {question_section_mapping["4b"]} section of the paper you are reviewing:\n' + save_dict[question_section_mapping["4b"]] + \
                    "\nBased on this section, do the authors use existing assets (e.g., code, data, models) that are not new contributions in their paper?  Please quote a short portion of each section directly as evidence before answering, but first start your answer with \"Yes\", \"No\", or \"Unsure\". "

    q4b_prompt_2_2 = "\nBased on the section, do the authors mention the licenses of the existing assets that they use? Please quote a short portion of each section directly as evidence before answering.\nQuote: "

    q4b_prompt_3 = f'The following is the {question_section_mapping["4b"]} section of the paper you are reviewing:\n{question_section_mapping["4b"]} section: ' + save_dict[question_section_mapping["4b"]] + \
                "\nBased on this section, if the authors are using existing assets (e.g., code, data, models), do they mention the licenses of the existing assets that they use?"

    q4c_prompt_1_1 = "The following is the abstract section of the paper you are reviewing:\n" + abstract + \
                    "\nBased on this abstract, do the authors curate/release new assets (e.g., code, data, models)? Please start your answer with \"Yes\", \"No\", or \"Unsure\"."

    q4c_prompt_1_2 = f'The following is the {question_section_mapping["4c"]} section of the paper you are reviewing:\n' + save_dict[question_section_mapping["4c"]] + \
                    "\nBased on this section, do the authors include any new assets either in the supplemental material or as a URL?"

    q4c_prompt_2_1 = "The following is the introduction section of the paper you are reviewing:\n" + introduction + \
                    "\nBased on the introduction, do the authors curate/release new assets (e.g., code, data, models) that are their own contribution? Please start your answer with \"Yes\", \"No\", or \"Unsure\"."

    q4c_prompt_2_2 = f'The following is the {question_section_mapping["4c"]} section of the paper you are reviewing:\n' + save_dict[question_section_mapping["4c"]] + \
                    "\nBased on this section, do the authors include any new assets either in the supplemental material or as a URL?"

    q4c_prompt_3 = f'The following is the {question_section_mapping["4c"]} section of the paper you are reviewing:\n' + save_dict[question_section_mapping["4c"]] + \
                    "\nBased on this section, do the authors include any new assets they curate/release either in the supplemental material or as a URL? If there are no new assets, please say so instead."

    q4d_prompt_1_1 = "The following is the abstract section of the paper you are reviewing:\n" + abstract + \
                    "\nBased on this abstract, do the authors curate/release new assets (e.g., code, data, models)? Please start your answer with \"Yes\", \"No\", or \"Unsure\"."

    q4d_prompt_1_2 = f'The following is the {question_section_mapping["4d"]} section of the paper you are reviewing:\n' + save_dict[question_section_mapping["4d"]] + \
                    "\nBased on this section, do the authors discuss whether and how consent was obtained from people whose data the authors are using/curating?"

    q4d_prompt_2_1 = "The following is the introduction section of the paper you are reviewing:\n" + introduction + \
                    "\nBased on the introduction, do the authors curate/release new assets (e.g., code, data, models) that are their own contribution? Please start your answer with \"Yes\", \"No\", or \"Unsure\"."

    q4d_prompt_2_2 = f'The following is the {question_section_mapping["4d"]} section of the paper you are reviewing:\n' + save_dict[question_section_mapping["4d"]] + \
                    "\nBased on this section, do the authors discuss whether and how consent was obtained from people whose data the authors are using/curating? If the authors are not using human data, please say so instead."

    q4d_prompt_3 = f'The following is the {question_section_mapping["4d"]} section of the paper you are reviewing:\n' + save_dict[question_section_mapping["4d"]] + \
                    "\nBased on this section, do the authors discuss whether and how consent was obtained from people whose data the authors are using/curating? If there are no new assets, or if the authors are not using human data, please say so instead."

    q4e_prompt_1_1 = "The following is the abstract section of the paper you are reviewing:\n" + abstract + \
                    "\nBased on this abstract, do the authors curate/release new assets (e.g., code, data, models)? Please start your answer with \"Yes\", \"No\", or \"Unsure\"."

    q4e_prompt_1_2 = f'The following is the {question_section_mapping["4e"]} section of the paper you are reviewing:\n' + save_dict[question_section_mapping["4e"]] + \
                    "\nBased on this section, do the authors discuss whether the data used/curated contains personally identifiable information or offensive content?"

    q4e_prompt_2_1 = "The following is the introduction section of the paper you are reviewing:\n" + introduction + \
                    "\nBased on the introduction, do the authors curate/release new assets (e.g., code, data, models) that are their own contribution? Please start your answer with \"Yes\", \"No\", or \"Unsure\"."

    q4e_prompt_2_2 = f'The following is the {question_section_mapping["4e"]} section of the paper you are reviewing:\n' + save_dict[question_section_mapping["4e"]] + \
                    "\nBased on this section, do the authors discuss whether the data used/curated contains personally identifiable information or offensive content?"

    q4e_prompt_3 = f'The following is the {question_section_mapping["4e"]} section of the paper you are reviewing:\n' + save_dict[question_section_mapping["4e"]] + \
                    "\nBased on this section, do the authors discuss whether the data used/curated contains personally identifiable information or offensive content? If there are no new assets, please say so instead."

    q5a_prompt_1_1 = "The following is the abstract of the paper you are reviewing:\n" + abstract + \
                    "\nBased on this abstract, do the authors use crowdsourcing or conduct research with human subjects? Please start your answer with \"Yes\", \"No\", or \"Unsure\"."

    q5a_prompt_1_2 = f'The following is the {question_section_mapping["5a"]} section of the paper you are reviewing:\n' + save_dict[question_section_mapping["5a"]] + \
                    "\nBased on this section, do the authors include the full text of instructions given to participants and screenshots, if applicable?"

    q5a_prompt_2_1 = "The following is the introduction of the paper you are reviewing:\n" + introduction + \
                    "\nBased on the introduction, do the authors use crowdsourcing or conduct research with human subjects? Please start your answer with \"Yes\", \"No\", or \"Unsure\"."

    q5a_prompt_2_2 = f'The following is the {question_section_mapping["5a"]} section of the paper you are reviewing:\n' + save_dict[question_section_mapping["5a"]] + \
                    "\nBased on this section, do the authors include the full text of instructions given to participants and screenshots, if applicable?"

    q5a_prompt_3 = f'The following is the {question_section_mapping["5a"]} section of the paper you are reviewing:\n' + save_dict[question_section_mapping["5a"]] + \
                    "\nBased on the section, if the authors use crowdsourcing or conduct research with human subjects, do the authors include the full text of instructions given to participants and screenshots, if applicable? If they do not use crowdsourcing or human subject research, please say so instead."

    q5b_prompt_1_1 = "The following is the abstract of the paper you are reviewing:\n" + abstract + \
                    "\nBased on this abstract, do the authors use crowdsourcing or conduct research with human subjects? Please start your answer with \"Yes\", \"No\", or \"Unsure\"."

    q5b_prompt_1_2 = f'The following is the {question_section_mapping["5b"]} section of the paper you are reviewing:\n' + save_dict[question_section_mapping["5b"]] + \
                    "\nBased on this section, do the authors describe any potential participant risks, with links to Institutional Review Board (IRB) approvals, if applicable?"

    q5b_prompt_2_1 = "The following is the introduction of the paper you are reviewing:\n" + introduction + \
                    "\nBased on the introduction, do the authors use crowdsourcing or conduct research with human subjects? Please start your answer with \"Yes\", \"No\", or \"Unsure\"."

    q5b_prompt_2_2 = f'The following is the {question_section_mapping["5b"]} section of the paper you are reviewing:\n' + save_dict[question_section_mapping["5b"]] + \
                    "\nBased on this section, do the authors describe any potential participant risks, with links to Institutional Review Board (IRB) approvals, if applicable?"

    q5b_prompt_3 = f'The following is the {question_section_mapping["5b"]} section of the paper you are reviewing:\n' + save_dict[question_section_mapping["5b"]] + \
                    "\nBased on the section, if the authors use crowdsourcing or conduct research with human subjects, do the authors describe any potential participant risks, with links to Institutional Review Board (IRB) approvals, if applicable? If they do not use crowdsourcing or human subject research, please say so instead."

    q5c_prompt_1_1 = "The following is the abstract of the paper you are reviewing:\n" + abstract + \
                    "\nBased on this abstract, do the authors use crowdsourcing or conduct research with human subjects? Please start your answer with \"Yes\", \"No\", or \"Unsure\"."

    q5c_prompt_1_2 = f'The following is the {question_section_mapping["5c"]} section of the paper you are reviewing:\n' + save_dict[question_section_mapping["5c"]] + \
                    "\nBased on this section, do the authors include the estimated hourly wage paid to participants and the total amount spent on participant compensation?"

    q5c_prompt_2_1 = "The following is the introduction of the paper you are reviewing:\n" + introduction + \
                    "\nBased on the introduction, do the authors use crowdsourcing or conduct research with human subjects? Please start your answer with \"Yes\", \"No\", or \"Unsure\"."

    q5c_prompt_2_2 = f'The following is the {question_section_mapping["5c"]} section of the paper you are reviewing:\n' + save_dict[question_section_mapping["5c"]] + \
                    "\nBased on this section, do the authors include the estimated hourly wage paid to participants and the total amount spent on participant compensation?"

    q5c_prompt_3 = f'The following is the {question_section_mapping["5c"]} section of the paper you are reviewing:\n' + save_dict[question_section_mapping["5c"]] + \
                "\nBased on the section, if the authors use crowdsourcing or conduct research with human subjects, do the authors include the estimated hourly wage paid to participants and the total amount spent on participant compensation? If they do not use crowdsourcing and do not use human subject research, please say so instead."
    
    problems_dict = {
        "q1c": [[q1c_prompt_1], [q1c_prompt_2], [q1c_prompt_3]],
        "q1b": [[q1b_prompt_1], [q1b_prompt_2], [q1b_prompt_3]],
        "q2a": [[q2a_prompt_1_1, q2a_prompt_1_2], [q2a_prompt_2_1, q2a_prompt_2_2], [q2a_prompt_3]],
        "q2b": [[q2b_prompt_1_1, q2b_prompt_1_2], [q2b_prompt_2_1, q2b_prompt_2_2], [q2b_prompt_3]],
        "q3a": [[q3a_prompt_1], [q3a_prompt_2], [q3a_prompt_3]],
        "q3b": [[q3b_prompt_1], [q3b_prompt_2], [q3b_prompt_3]],
        "q3c": [[q3c_prompt_1], [q3c_prompt_2], [q3c_prompt_3]],
        "q3d": [[q3d_prompt_1], [q3d_prompt_2], [q3d_prompt_3]],
        "q4a": [[q4a_prompt_1_1, q4a_prompt_1_2], [q4a_prompt_2_1, q4a_prompt_2_2], [q4a_prompt_3]],
        "q4b": [[q4b_prompt_1_1, q4b_prompt_1_2], [q4b_prompt_2_1, q4b_prompt_2_2], [q4b_prompt_3]],
        "q4c": [[q4c_prompt_1_1, q4c_prompt_1_2], [q4c_prompt_2_1, q4c_prompt_2_2], [q4c_prompt_3]],
        "q4d": [[q4d_prompt_1_1, q4d_prompt_1_2], [q4d_prompt_2_1, q4d_prompt_2_2], [q4d_prompt_3]],
        "q4e": [[q4e_prompt_1_1, q4e_prompt_1_2], [q4e_prompt_2_1, q4e_prompt_2_2], [q4e_prompt_3]],
        "q5a": [[q5a_prompt_1_1, q5a_prompt_1_2], [q5a_prompt_2_1, q5a_prompt_2_2], [q5a_prompt_3]],
        "q5b": [[q5b_prompt_1_1, q5b_prompt_1_2], [q5b_prompt_2_1, q5b_prompt_2_2], [q5b_prompt_3]],
        "q5c": [[q5c_prompt_1_1, q5c_prompt_1_2], [q5c_prompt_2_1, q5c_prompt_2_2], [q5c_prompt_3]]
    }

    # put the inputs into input format for openAI API
    for problem in problems_dict.keys(): # e.g., "q1c"
        for alternative in problems_dict[problem]: # e.g., [q1c_prompt_1]
            for ministep_index in range(len(alternative)): # e.g., q1c_prompt_1
                alternative[ministep_index] = turn_into_input_format("user", alternative[ministep_index])

    return problems_dict

In [ ]:
problems_dict = generate_prompts(question_section_mapping)

In [11]:
def query_problem(system_prompt, alternative, model="gpt-4", max_tokens=150, temperature=1, top_p=1, n=1, verbose=False):
    # alternative is a list of prompts for the same prompting strategy on the same question
    messages = [system_prompt, alternative[0]]
    if verbose:
        print("messages:", messages)
        print("temperature", temperature, "top_p", top_p, "n", n)
    
    completion1 = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        max_tokens=max_tokens,
        temperature=temperature,
        top_p=top_p, 
        n=n
    )
    
    if completion1["usage"]["prompt_tokens"] > 8000:
        raise ValueError("too many input tokens.")
    
    if len(alternative) == 1:
        return [completion1["choices"]]

    # figure out majority answer
    num_yes = 0
    yes_index = None
    for choice in completion1["choices"]:
        returned_text = choice["message"]["content"]
        if returned_text[:3] == "Yes" or returned_text[:6] == "Unsure":
            num_yes += 1
            yes_index = choice["index"]
        elif returned_text[:2] != "No":
            print("model did not return a valid answer.", returned_text[:30])
        if choice["finish_reason"] != "stop":
            print("model did not finish its answer under token limit.", returned_text[-30:])
        if choice["message"]["role"] != "assistant":
            raise ValueError("model answered in the wrong role.")
        if verbose:
            print("index", choice["index"], "returned_text:", returned_text)
    
    if verbose:
        print("num_yes:", num_yes)
        print("yes_index:", yes_index)
        
    if num_yes >= 2:
        messages = [system_prompt, alternative[0], completion1["choices"][yes_index]["message"], alternative[1]]
        if verbose:
            print("messages:", messages)
        
        completion2 = openai.ChatCompletion.create(
            model=model,
            messages=messages,
            max_tokens=max_tokens,
            temperature=temperature,
            top_p=top_p, 
            n=n
        )

        if completion2["usage"]["prompt_tokens"] > 8000:
            raise ValueError("too many input tokens.")
        
        for choice in completion2["choices"]:
            returned_text = choice["message"]["content"]
            if choice["finish_reason"] != "stop":
                print("model did not finish its answer under token limit.", returned_text[-30:])
            if choice["message"]["role"] != "assistant":
                raise ValueError("model answered in the wrong role.")
            if verbose:
                print("index", choice["index"], "returned_text:", returned_text)
        
        return [completion1["choices"], completion2["choices"]]

    return [completion1["choices"]]

In [12]:
import time

def pilot(verbose=False):
    # first, pick a few temperature/top_p sampling candidates 
    # that do the best on n=3 generations on a subset of the prompts.
    pilot_questions = ["q1c", "q2a", "q3a", "q4a", "q5a"]
    for problem in pilot_questions:
        # select just the first prompt (simplest)
        alternative = problems_dict[problem][0]
        for i in range(0, 21, 1):
            temperature = i/10
            answers = query_problem(system_prompt, alternative, temperature=temperature, n=3, verbose=verbose)
            # save the answers to a file
            with open(f"pilot_data_hyp/{problem}/pilot_answers_temp_{temperature}_{problem}.json", "w") as f:
                json.dump(answers, f)
            time.sleep(10) # OpenAI limit is like 40000 tokens/min
        
        for i in range(0, 11, 1):
            top_p = i/10
            answers = query_problem(system_prompt, alternative, top_p = top_p, n=3, verbose=verbose)
            # save the answers to a file
            with open(f"pilot_data_hyp/{problem}/pilot_answers_topp_{top_p}_{problem}.json", "w") as f:
                json.dump(answers, f)
            time.sleep(10) # OpenAI limit is like 40000 tokens/min    

In [13]:
pilot(verbose=True)

In [14]:
def print_output(problem, temperature=None, top_p=None):
    if top_p is None:
        if len(f"{temperature}") == 1:
            temperature = f"{temperature}.0"
        with open(f"pilot_data_hyp/{problem}/pilot_answers_temp_{temperature}_{problem}.json", "r") as f:
           answers = json.load(f)
    
    if temperature is None:
        if len(f"{top_p}") == 1:
            top_p = f"{top_p}.0"
        with open(f"pilot_data_hyp/{problem}/pilot_answers_topp_{top_p}_{problem}.json", "r") as f:
            answers = json.load(f)

    for i in range(len(answers[0])):
        for j in range(len(answers)):
            print(answers[j][i]["message"]["content"], "\n")

In [ ]:
print_output("q5a", top_p=0.9)

If we optimize the system prompt jointly with the prompts:
2 pairs of hyperparameters * 3 system prompts * 3 generations * 3 prompt alternatives * 16 questions = 864 labels -> too many

Instead, we make the assumption that system prompts will not change much for different questions in the same category (same assumption made above):
2 pairs of hyperparameters * 3 system prompts * 3 generations * 3 prompt alternatives * 5 questions = 270 labels

We make another assumption that the system prompts will not interact much with the hyperparameter selection, and take t=1 (the most volatile case) to use:
1 hyperparameter * 3 system prompts * 3 generations * 3 prompt alternatives * 5 questions = 135 labels

Using this, we will see if there is significant difference between different system prompts, or pairings of system prompt + alternative that work well together. "testing for artifacts"